Welcome to your DataCamp project audition! This notebook must be filled out and vetted before a contract can be signed and you can start creating your project.

The first step is forking the repository in which this notebook lives. After that, there are two parts to be completed in this notebook:

- **Project information**:  The title of the project, a project description, etc.

- **Project introduction**: The three first text and code cells that will form the introduction of your project.

When complete, please email the link to your forked repo to projects@datacamp.com with the email subject line _DataCamp project audition_. If you have any questions, please reach out to projects@datacamp.com.

# Project information

**Project title**: Check your lab QC data! Is everything in limit?  

**Name:** Deniz Ilhan Topcu

**Email address associated with your DataCamp account:** ditopcu@gmail.com

**GitHub username:** ditopcu

**Project description**: 

How can clinical laboratories be sure about their results? Statistical **Quality Control** (QC) methods are helping laboratories every day for ensuring the required quality. But evaluating lots of quality control results can be overwhelming. Data science tools could help us to facilitate this process, once again!  
In this project, we will investigate laboratory QC data using data import, data wrangling, and visualization tools in R. We’re going to calculate QC statistics and plot Levey-Jennings charts and apply basic [_“Westgard Rules”._  ](https://www.westgard.com/mltirule.htm)

Completing [“Introduction to the Tidyverse course” ](https://www.datacamp.com/courses/introduction-to-the-tidyverse)
or experience with `dplyr` and `ggplot2` packages is recommended. Also, familiarity with data importing, writing functions and basic functional programming skills will be helpful.  
  
  
We will examine two level QC results for 10 different biochemistry analytes in this project. The dataset comprises simulated QC results based on real clinical laboratory data.  

# Project introduction

***Note: nothing needs to be filled out in this cell. It is simply setting up the template cells below.***

The final output of a DataCamp project looks like a blog post: pairs of text and code cells that tell a story about data. The text is written from the perspective of the data analyst and *not* from the perspective of an instructor on DataCamp. So, for this blog post intro, all you need to do is pretend like you're writing a blog post -- forget the part about instructors and students.

Below you'll see the structure of a DataCamp project: a series of "tasks" where each task consists of a title, a **single** text cell, and a **single** code cell. There are 8-12 tasks in a project and each task can have up to 10 lines of code. What you need to do:
1. Read through the template structure.
2. As best you can, divide your project as it is currently visualized in your mind into tasks.
3. Fill out the template structure for the first three tasks of your project.

As you are completing each task, you may wish to consult the project notebook format in our [documentation](https://instructor-support.datacamp.com/projects/datacamp-projects-jupyter-notebook). Only the `@context` and `@solution` cells are relevant to this audition.

## 1. Understanding and Importing Data

Clinical laboratory tests such as serum glucose, HDL, Iron, etc. are using to evaluate the condition of a patient and they are important for clinical decision.  

![Clinical Lab](img/visual.png)


Quality control (QC) is a process to periodically examine these tests measurement procedure and verify that it is performing according to predefined specifications. There are two main errors for measurement: inaccuracy and imprecision.  


![Measure](img/measure03.png)


**Accuracy** is used to describe the closeness of a measurement to the true value.  **Precision** is the closeness of agreement between repeated measurements of a sample.  
Laboratories are using internal quality control (IQC) procedures to assess their imprecision (random error). Traditionally, IQC uses sample materials with assigned values and IQC results are evaluated continuously in relation to these known values.   

To understand these procedures and evaluate imprecision, we are going to inspect QC results. 
We have QC results for 10 different analytes as separate CSV files and one CSV file for predefined specifications for these 10 tests. We are going to import these files, then evaluate precision as in terms of mean, standard deviation. Finally, we plot Levey-Jennings charts using these statistics.  

Let's start with data importing.


In [ ]:
# Load tidyverse and lubridate package
library(tidyverse)
library(lubridate)


# Check current directory for .csv files with ends "_QC_results"
list.files("datasets/", pattern = "_QC_results.csv")

# Test import: import Albumin_QC_results.csv file into test_read using read_csv and inspect results.
test_read <- read_csv("datasets/Albumin_QC_results.csv")

glimpse(test_read)
head(test_read)



# Import again  Albumin_QC_results.csv file now into glucose_qc with datetime parsing and
#  factor conversion


albumin_qc <- read_csv("datasets/Albumin_QC_results.csv") %>%
  mutate(result_date = ymd_hms(result_date)) %>%
  mutate(lot_number = as.factor(lot_number), level = as.factor(level))

#Inspect results
glimpse(albumin_qc)
head(albumin_qc)


# Import all test specifications(test_specs.csv) into test_performance_data and inspect
test_performance_data <- read_csv("datasets/test_specs.csv")

glimpse(test_performance_data)


## 2. Importing Data Using Function

We have just read glucose QC results. But there are 9 files to go! We want to evaluate all biochemistry data.  
Should we copy&past above code?

No. There must be more elegant way. Let's define a function and use this function to read all files. To do this:

* Define a reader function: We need a function which takes file name as parameter and return a tibble. While defining function we should add col types (col_types) while using read_csv file in a function. This will a) Ensure we are reading files correctly b) Suppress warning messages

* Test function

* Use some kind of loops to read all files: We will use purr::map family functions

Now we are going to complete these steps.

In [ ]:
# Convert above code into a function which accepts file_name and returns tibble

qc_result_reader <- function(file_name) {
  read_csv(file_name,
    col_types = cols(
      new_device_name = col_character(),
      lot_number = col_integer(),
      level = col_character(),
      result_date = col_character(),
      test_code = col_character(),
      result = col_double(),
      unit = col_character(),
      name = col_character()
    )
  ) %>%
    mutate(result_date = ymd_hms(result_date)) %>%
    mutate(lot_number = as.factor(lot_number), level = as.factor(level))
}


# Test function with Glucose_QC_results.csv and import glucose data into albumin_qc

glucose_qc <- qc_result_reader("datasets/Glucose_QC_results.csv")


glimpse(glucose_qc)
head(glucose_qc)

## 3. Importing Multiple Files

Yes! Our function is working!  Now we have a function to read our QC Files. 

_Hint:_ We provide `col_types` to ensure we are reading columns correctly. We can also use specific `col_` instead of additional  mutate's. For example: `result_date = col_datetime(format = "%Y.%m.%d %H:%M:%S")` instead of `lubridate::ymd_hms`. This can be fast in big data sets.  
  
We can use `map_df` from `purrr` package to read all files and combine into one single tibble. `purrr::map` family functions provides simple interfaces for repetitive tasks. 


`map_df` function also accepts `.id` parameter to create additional variable which contains index name. In this case our file name.  (use ?map_df for details)

To obtain test names from file names we can use regex.

After that we are ready to do real analysis...

In [ ]:

# Get file names and paths using full.names argument

file_list <- list.files(path = "datasets/", pattern = "_QC_results.csv", full.names = TRUE)

# Inspect file_list
file_list


# use regex to remove path and "_QC_results" tag and obtain test names. Set these  names to use in map_df
test_names <- file_list %>%
  str_extract("(?<=\\/)(.*?)(?=\\.)") %>% # extracts text between / and .
  str_replace("_QC_results", "") # removes _QC_results

names(file_list) <- test_names

# Read all files in file_list
all_test_qc <- map_df(file_list, qc_result_reader, .id = "test_name")

# inspect data
glimpse(all_test_qc)
head(all_test_qc)

# count test and levels
count(all_test_qc, test_name, level)


# Now we are ready to join data and do some statistics.

*Stop here! Only the three first tasks. :)*